In [ ]:
import pandas as pd


# Read the CSV file into a DataFrame
df = pd.read_csv('data.csv')


In [ ]:
import pandas as pd
#df = df.iloc[:, :-6]

# Calculate the 'number of study hours' attribute
df['number of study hours'] = df['Study hours end time'] - df['Study hours start time']

# Remove the old attributes
df = df.drop(columns=['Study hours start time', 'Study hours end time'])



In [ ]:
# Assuming df is your DataFrame
print(df.columns)

In [ ]:
# Write the updated DataFrame back to the CSV file
df.to_csv('data.csv', index=False)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


# Define features and target variables
features = df[['In which year of your studies are you currently enrolled:',
               'BAC specialty', 'Where do you prefer to study:',
               'You prefer to study alone:', 'Do you use a planer or a schedule',
               'Problems', '2. Second:', '3. Third:', '4. Forth:', '5. Fifth:',
               '6. sixth:', '7. seventh:', '8. eighth',
               'Rate your satisfaction with the schools program:',
               'Do you engage in any external activities (not related to the study field):',
               'Your level of motivation for joining ENSIA',
               'How much do you like the field of Artificial Intelligence :',
               'How do others feedbacks influence you:', 'How many hours do you sleep',
               'How well do you handle stress and pressure :', 'English level',
               'Programming level', 'Computer science level', 'Mathematics level',
               'Books', 'Lectures', 'Online documents', 'Online tutorials', 'Private courses',
               'YouTube videos', 'number of study hours']]
target_variables = df[['1Y_Intensive english', '1Y_Seminar', '1Y_CPSCT', '1Y_Analysis I',
                       '1Y_Algebra', '1Y_A. Com. I', '1Y_Intro. Prog. C++', '1Y_Analysis II',
                       '1Y_A. Com. II', '1Y_OOP', '1Y_Linear Algebra', '1Y_Proba Stat',
                       '2Y_Analysis III', '2Y_DSA', '2Y_Complexity', '2Y_Proba Stat II',
                       '2Y_Archi 1', '2Y_Logic', '2Y_Archi II', '2Y_Oper. res',
                       '2Y_Lin. Alg II', '2Y_DataBases', '2Y_Intro AI', '2Y_Web Prog.',
                       '2Y_Soft. Eng.', '2Y_Oper. Syst', '2Y_Intro. Busin.', 'Average']]

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target_variables, test_size=0.2, random_state=42)
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Train the Random Forest regressor
rf_regressor = RandomForestRegressor(n_estimators=1000, random_state=42)
rf_regressor.fit(X_train, y_train)

# Predict target variables for the training and testing sets
y_train_pred = rf_regressor.predict(X_train)
y_test_pred = rf_regressor.predict(X_test)

# Calculate regression metrics for training set
train_metrics = {}
train_metrics['MAE'] = mean_absolute_error(y_train, y_train_pred)
train_metrics['MSE'] = mean_squared_error(y_train, y_train_pred)
train_metrics['R-squared'] = r2_score(y_train, y_train_pred)

# Calculate regression metrics for testing set
test_metrics = {}
test_metrics['MAE'] = mean_absolute_error(y_test, y_test_pred)
test_metrics['MSE'] = mean_squared_error(y_test, y_test_pred)
test_metrics['R-squared'] = r2_score(y_test, y_test_pred)

# Print metrics
print("Training set metrics:")
for metric, value in train_metrics.items():
    print(f"{metric}: {value}")

print("\nTesting set metrics:")
for metric, value in test_metrics.items():
    print(f"{metric}: {value}")


In [ ]:
import joblib
# Save the trained model to a file
joblib.dump(rf_regressor, 'rf_regressor_model.pkl')

# Optionally, save the feature names if needed later
feature_names = features.columns
joblib.dump(feature_names, 'feature_names.pkl')

In [ ]:
# Assuming you have a new instance stored in a DataFrame called 'new_instance'
new_instance = pd.DataFrame({
    'In which year of your studies are you currently enrolled:': [2],
    'BAC specialty': [2],
    'Where do you prefer to study:': [4],
    'You prefer to study alone:': [1],
    'Do you use a planer or a schedule': [0],
    'Problems': [1],
    '2. Second:': [6],
    '3. Third:': [7],
    '4. Forth:': [3],
    '5. Fifth:': [4],
    '6. sixth:': [8],
    '7. seventh:': [2],
    '8. eighth': [6],
    'Rate your satisfaction with the schools program:': [5],
    'Do you engage in any external activities (not related to the study field):': [0],
    'Your level of motivation for joining ENSIA': [8],
    'How much do you like the field of Artificial Intelligence :': [9],
    'How do others feedbacks influence you:': [7],
    'How many hours do you sleep': [7],
    'How well do you handle stress and pressure :': [8],
    'English level': [4],
    'Programming level': [3],
    'Computer science level': [2],
    'Mathematics level': [1],
    'Books': [1],
    'Lectures': [0],
    'Online documents': [0],
    'Online tutorials': [1],
    'Private courses': [1],
    'YouTube videos': [0],
    'number of study hours':[15]
})

# Use the trained RandomForestClassifier to make predictions
predicted_results = rf_regressor.predict(new_instance)

# Print the predicted results
print("Predicted Results:")
for i, target_variable in enumerate(target_variables.columns):
    print(f"{target_variable}: {predicted_results[0, i]}")


In [ ]:
import shap

# Calculate SHAP values for the "average" target variable only
explainer = shap.Explainer(rf_regressor, X_train)
shap_values = explainer.shap_values(X_test)

# Find the index of the "average" target variable
average_index = list(target_variables.columns).index("Average")

# Get SHAP values for the "average" target variable only
shap_values_average = shap_values[average_index]

# Get feature names
feature_names = X_train.columns

# Get the average SHAP values for each feature
average_shap_values = np.mean(np.abs(shap_values_average), axis=0)

# Get the indices of the top three features with the highest absolute SHAP values
top_three_indices = np.argsort(average_shap_values)[::-1][:3]

# Print the three most important features
print("Top three most important features for Average:")
for index in top_three_indices:
    feature_name = feature_names[index]
    shap_val = average_shap_values[index]
    print(f"{feature_name}: {shap_val}")

# Make recommendations based on the three most important features
print("Recommendations based on the three most important features:")
# Provide specific recommendations based on the identified important features for the "average" target variable.


In [ ]:
import numpy as np

new_instance_array = np.array([1, 2, 2, 4, 1, 0, 1, 6, 7, 3, 4, 8, 2, 6, 5, 0, 8, 9, 7, 8, 4, 3, 2, 1, 1, 0, 0, 1, 1, 0, 15])


In [ ]:
from sklearn.neighbors import NearestNeighbors

# Train the KNN model using all the data
k = 50  # Number of nearest neighbors
knn_model = NearestNeighbors(n_neighbors=k)
knn_model.fit(features)

def find_nearest_neighbors_average(new_instance):
    # Find the 50 nearest neighbors for the new instance
    distances, indices = knn_model.kneighbors([new_instance])
    nearest_neighbor_indices = indices[0]  # Get the indices of nearest neighbors
    nearest_neighbors_average = target_variables.iloc[nearest_neighbor_indices, :]['Average'].tolist()  # Get the 'Average' values for nearest neighbors
    return nearest_neighbors_average

# Example of using the function with a new instance
neighbors_avg = find_nearest_neighbors_average(new_instance_array)
print("Values of 'Average' for 50 nearest neighbors:", neighbors_avg)


In [ ]:
from joblib import dump

# Save the trained NearestNeighbors model
model_filename = "knn_model.joblib"
dump(knn_model, model_filename)
print("Model saved as", model_filename)
